In [1]:
import pandas as pd
import sqlite3
# import requests
# import zipfile
# import io
# from google.cloud import bigquery
# data from 1992 to 2018

In [2]:
df = pd.read_csv('ca_fire.csv')
df.head()

,date,time,cause_class,cause,fire_size,fire_size_class,lat,long,elevation,county,...,diameter,tree_alive,invasive_sampling,co_mean,temp_mean,humidity_mean,wind_direction_mean,wind_speed_mean,month,day_of_year
0,2001-01-05,1944,human,missing data/not specified/undetermined,80.0,c,36.203431,-119.037519,11300,tulare county,...,24.4,1,1,1.909058,48.393940,46.333334,149.172348,3.659470,1,5
1,2001-05-08,2059,human,arson/incendiarism,0.1,a,36.217550,-119.055739,11300,tulare county,...,24.4,1,1,0.476812,75.458333,35.000000,195.322916,4.881250,5,128
2,2001-05-23,2000,human,equipment and vehicle use,0.1,a,36.464731,-118.984781,11300,tulare county,...,24.4,1,1,0.437953,76.677083,35.875000,219.322917,4.908333,5,143
3,2001-06-01,1515,human,equipment and vehicle use,2.0,b,36.478800,-119.018250,11300,tulare county,...,24.4,1,1,0.397282,78.052083,36.062500,218.927083,5.421875,6,152
4,2001-06-03,1750,human,equipment and vehicle use,10.0,c,36.507739,-118.907731,11300,tulare county,...,24.4,1,1,0.249185,60.833333,42.583333,248.416667,5.121875,6,154


In [3]:
# # url for zipped data
# url = 'https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.6/RDS-2013-0009.6_SQLITE.zip'
# # get zipped data
# response = requests.get(url, stream=True)
# z = zipfile.ZipFile(io.BytesIO(response.content))

In [4]:
# # extract necessary files
# z.extract('Data/FPA_FOD_20221014.sqlite')
# z.extract('Data/_variable_descriptions.csv')

In [5]:
# original pandas_gbq query
long_sql_aq = """
SELECT poc,latitude,longitude,parameter_name,sample_duration,
date_local,units_of_measure,observation_count,
arithmetic_mean,county_code,site_num
FROM `bigquery-public-data.epa_historical_air_quality.co_daily_summary`
WHERE state_code = '06' and date_local < '2019-01-01' and date_local > '1996-12-31' and datum != 'UNKNOWN' and datum != 'NAD27'
UNION ALL
SELECT poc,latitude,longitude,parameter_name,sample_duration,
date_local,units_of_measure,observation_count,
arithmetic_mean,county_code,site_num
FROM `bigquery-public-data.epa_historical_air_quality.pressure_daily_summary`
WHERE state_code = '06' and date_local < '2019-01-01' and date_local > '1996-12-31' and datum != 'UNKNOWN' and datum != 'NAD27'
UNION ALL
SELECT poc,latitude,longitude,parameter_name,sample_duration,
date_local,units_of_measure,observation_count,
arithmetic_mean,county_code,site_num
FROM `bigquery-public-data.epa_historical_air_quality.rh_and_dp_daily_summary`
WHERE state_code = '06' and date_local < '2019-01-01' and date_local > '1996-12-31' and datum != 'UNKNOWN' and datum != 'NAD27'
UNION ALL
SELECT poc,latitude,longitude,parameter_name,sample_duration,
date_local,units_of_measure,observation_count,
arithmetic_mean,county_code,site_num
FROM `bigquery-public-data.epa_historical_air_quality.temperature_daily_summary`
WHERE state_code = '06' and date_local < '2019-01-01' and date_local > '1996-12-31' and datum != 'UNKNOWN' and datum != 'NAD27'
UNION ALL
SELECT poc,latitude,longitude,parameter_name,sample_duration,
date_local,units_of_measure,observation_count,
arithmetic_mean,county_code,site_num
FROM `bigquery-public-data.epa_historical_air_quality.wind_daily_summary`
WHERE state_code = '06' and date_local < '2019-01-01' and date_local > '1996-12-31' and datum != 'UNKNOWN' and datum != 'NAD27'
"""

long_sql_pt = """
SELECT  
    measurement_year,
    tree_county_code,latitude,longitude,elevation,
    trees_per_acre_unadjusted,
    water_on_plot_code_name,
    species_common_name,species_group_code_name,
    total_height,
    current_diameter,
    tree_status_code_name,
    invasive_sampling_status_code_name,
FROM `bigquery-public-data.usfs_fia.plot_tree`
WHERE plot_state_code = 6 AND total_height > 0 AND measurement_year > 1996
"""


In [6]:
# test_aq = bigquery.Client().query(long_sql_aq,project='bigquery-public-data').to_dataframe()
# test_aq

In [7]:
sql_aq = """
SELECT *
FROM `california_air_trees_fires.epa_aq_ca5`
"""

sql_pt = """
SELECT *
FROM `california_air_trees_fires.usfs_fia_pt_ca2`
"""

## Air Quality

In [8]:
# pandas gbq will auth with google account first, rerun after
aq = pd.read_gbq(sql_aq, dialect='standard', project_id='my-ds-projects', use_bqstorage_api=True)

In [9]:
aq.date_local = aq.date_local.astype('datetime64[ns]')
aq

,poc,latitude,longitude,parameter_name,sample_duration,date_local,units_of_measure,observation_count,arithmetic_mean,county_code,site_num
0,1,39.202946,-122.018028,Wind Speed - Resultant,1 HOUR,1999-03-24,Knots,24,4.916667,011,0002
1,1,39.012396,-122.092471,Wind Speed - Resultant,1 HOUR,1999-03-13,Knots,24,3.791667,011,1001
2,1,39.012396,-122.092471,Wind Speed - Resultant,1 HOUR,1999-06-24,Knots,24,5.333333,011,1001
3,1,38.890941,-121.003373,Wind Speed - Resultant,1 HOUR,1999-07-07,Knots,24,3.083333,017,0020
4,1,38.890941,-121.003373,Wind Speed - Resultant,1 HOUR,1999-07-27,Knots,24,2.708333,017,0020
...,...,...,...,...,...,...,...,...,...,...,...
5150901,1,34.051110,-118.456360,Carbon monoxide,8-HR RUN AVG END HOUR,2015-11-08,Parts per million,24,0.362500,037,0113
5150902,1,34.051110,-118.456360,Carbon monoxide,8-HR RUN AVG END HOUR,2015-12-30,Parts per million,24,0.491667,037,0113
5150903,1,34.051110,-118.456360,Barometric pressure,1 HOUR,2018-04-19,Millibars,24,1005.666667,037,0113
5150904,1,34.051110,-118.456360,Barometric pressure,1 HOUR,2018-07-18,Millibars,24,1004.416667,037,0113


In [10]:
aq['year'] = aq.date_local.dt.year
aq.county_code = aq.county_code.astype(int)
aq

,poc,latitude,longitude,parameter_name,sample_duration,date_local,units_of_measure,observation_count,arithmetic_mean,county_code,site_num,year
0,1,39.202946,-122.018028,Wind Speed - Resultant,1 HOUR,1999-03-24,Knots,24,4.916667,11,0002,1999
1,1,39.012396,-122.092471,Wind Speed - Resultant,1 HOUR,1999-03-13,Knots,24,3.791667,11,1001,1999
2,1,39.012396,-122.092471,Wind Speed - Resultant,1 HOUR,1999-06-24,Knots,24,5.333333,11,1001,1999
3,1,38.890941,-121.003373,Wind Speed - Resultant,1 HOUR,1999-07-07,Knots,24,3.083333,17,0020,1999
4,1,38.890941,-121.003373,Wind Speed - Resultant,1 HOUR,1999-07-27,Knots,24,2.708333,17,0020,1999
...,...,...,...,...,...,...,...,...,...,...,...,...
5150901,1,34.051110,-118.456360,Carbon monoxide,8-HR RUN AVG END HOUR,2015-11-08,Parts per million,24,0.362500,37,0113,2015
5150902,1,34.051110,-118.456360,Carbon monoxide,8-HR RUN AVG END HOUR,2015-12-30,Parts per million,24,0.491667,37,0113,2015
5150903,1,34.051110,-118.456360,Barometric pressure,1 HOUR,2018-04-19,Millibars,24,1005.666667,37,0113,2018
5150904,1,34.051110,-118.456360,Barometric pressure,1 HOUR,2018-07-18,Millibars,24,1004.416667,37,0113,2018


In [11]:
aq.head()

,poc,latitude,longitude,parameter_name,sample_duration,date_local,units_of_measure,observation_count,arithmetic_mean,county_code,site_num,year
0,1,39.202946,-122.018028,Wind Speed - Resultant,1 HOUR,1999-03-24,Knots,24,4.916667,11,0002,1999
1,1,39.012396,-122.092471,Wind Speed - Resultant,1 HOUR,1999-03-13,Knots,24,3.791667,11,1001,1999
2,1,39.012396,-122.092471,Wind Speed - Resultant,1 HOUR,1999-06-24,Knots,24,5.333333,11,1001,1999
3,1,38.890941,-121.003373,Wind Speed - Resultant,1 HOUR,1999-07-07,Knots,24,3.083333,17,0020,1999
4,1,38.890941,-121.003373,Wind Speed - Resultant,1 HOUR,1999-07-27,Knots,24,2.708333,17,0020,1999


In [12]:
aq.year.value_counts().sort_index().index.max(),aq.year.value_counts().sort_index().index.min()

(2018, 1997)

## Tree Plots

In [13]:
# pandas gbq will auth with google account first, rerun after
pt = pd.read_gbq(sql_pt, dialect='standard', project_id='my-ds-projects', use_bqstorage_api=True)

In [14]:
pt.head()

,measurement_year,tree_county_code,latitude,longitude,elevation,trees_per_acre_unadjusted,water_on_plot_code_name,species_common_name,species_group_code_name,total_height,current_diameter,tree_status_code_name,invasive_sampling_status_code_name
0,2013,51,37.586124,-118.186569,10500,0.999188,None - no water sources within the accessible ...,Great Basin bristlecone pine,Other western softwoods,39,40.299999,Live tree,Invasive plant data collected on all accessibl...
1,2011,51,37.733501,-118.257523,9900,0.999188,None - no water sources within the accessible ...,limber pine,Other western softwoods,43,26.400000,Live tree,Invasive plant data collected on all accessibl...
2,2006,107,36.351498,-118.465668,10700,0.999188,Permanent streams or ponds too small to qualif...,foxtail pine,Other western softwoods,64,37.299999,Live tree,Not collecting invasive plant data
3,2006,107,36.511063,-118.471970,10900,6.018046,None - no water sources within the accessible ...,foxtail pine,Other western softwoods,29,8.900000,Live tree,Not collecting invasive plant data
4,2007,71,34.125198,-116.906303,10100,0.999188,None - no water sources within the accessible ...,limber pine,Other western softwoods,113,45.000000,Dead tree,Invasive plant data collected on all accessibl...


In [15]:
pt = pt[pt['tree_county_code'].notna()].copy()
pt.columns

Index(['measurement_year', 'tree_county_code', 'latitude', 'longitude',
       'elevation', 'trees_per_acre_unadjusted', 'water_on_plot_code_name',
       'species_common_name', 'species_group_code_name', 'total_height',
       'current_diameter', 'tree_status_code_name',
       'invasive_sampling_status_code_name'],
      dtype='object')

In [16]:
pt.head()

,measurement_year,tree_county_code,latitude,longitude,elevation,trees_per_acre_unadjusted,water_on_plot_code_name,species_common_name,species_group_code_name,total_height,current_diameter,tree_status_code_name,invasive_sampling_status_code_name
0,2013,51,37.586124,-118.186569,10500,0.999188,None - no water sources within the accessible ...,Great Basin bristlecone pine,Other western softwoods,39,40.299999,Live tree,Invasive plant data collected on all accessibl...
1,2011,51,37.733501,-118.257523,9900,0.999188,None - no water sources within the accessible ...,limber pine,Other western softwoods,43,26.400000,Live tree,Invasive plant data collected on all accessibl...
2,2006,107,36.351498,-118.465668,10700,0.999188,Permanent streams or ponds too small to qualif...,foxtail pine,Other western softwoods,64,37.299999,Live tree,Not collecting invasive plant data
3,2006,107,36.511063,-118.471970,10900,6.018046,None - no water sources within the accessible ...,foxtail pine,Other western softwoods,29,8.900000,Live tree,Not collecting invasive plant data
4,2007,71,34.125198,-116.906303,10100,0.999188,None - no water sources within the accessible ...,limber pine,Other western softwoods,113,45.000000,Dead tree,Invasive plant data collected on all accessibl...


In [17]:
pt.measurement_year.value_counts().sort_index().index.max(),pt.measurement_year.value_counts().sort_index().index.min()

(2018, 2001)

In [18]:
# map easy categorical values to 1 and 0
pt.tree_status_code_name = pt.tree_status_code_name.map({'Live tree':1,'Dead tree':0})
pt.invasive_sampling_status_code_name = pt.invasive_sampling_status_code_name.map({'Invasive plant data collected on all accessible land conditions':1,'Not collecting invasive plant data':0})

In [19]:
pt.head()

,measurement_year,tree_county_code,latitude,longitude,elevation,trees_per_acre_unadjusted,water_on_plot_code_name,species_common_name,species_group_code_name,total_height,current_diameter,tree_status_code_name,invasive_sampling_status_code_name
0,2013,51,37.586124,-118.186569,10500,0.999188,None - no water sources within the accessible ...,Great Basin bristlecone pine,Other western softwoods,39,40.299999,1,1
1,2011,51,37.733501,-118.257523,9900,0.999188,None - no water sources within the accessible ...,limber pine,Other western softwoods,43,26.400000,1,1
2,2006,107,36.351498,-118.465668,10700,0.999188,Permanent streams or ponds too small to qualif...,foxtail pine,Other western softwoods,64,37.299999,1,0
3,2006,107,36.511063,-118.471970,10900,6.018046,None - no water sources within the accessible ...,foxtail pine,Other western softwoods,29,8.900000,1,0
4,2007,71,34.125198,-116.906303,10100,0.999188,None - no water sources within the accessible ...,limber pine,Other western softwoods,113,45.000000,0,1


In [20]:
# Performed 9 aggregations grouped on columns: 'measurement_year', 'tree_county_code'
pt_group = pt.groupby(
    ['measurement_year', 'tree_county_code']
        ).agg(
            elevation_mean=('elevation', 'mean'), 
            trees_per_acre_mean=('trees_per_acre_unadjusted', 'mean'),
            most_common_water_source=('water_on_plot_code_name', lambda s: s.value_counts().index[0]),
            most_common_species=('species_common_name', lambda s: s.value_counts().index[0]),
            most_common_species_group=('species_group_code_name', lambda s: s.value_counts().index[0]),
            height_mean=('total_height', 'mean'),
            diameter_mean=('current_diameter', 'mean'),
            percent_trees_alive=('tree_status_code_name', 'mean'),
            percent_invasive_plant=('invasive_sampling_status_code_name', 'mean')
            ).reset_index()


In [21]:
pt_group.head()

,measurement_year,tree_county_code,elevation_mean,trees_per_acre_mean,most_common_water_source,most_common_species,most_common_species_group,height_mean,diameter_mean,percent_trees_alive,percent_invasive_plant
0,2001,3,8207.18232,8.794056,Permanent streams or ponds too small to qualif...,California red fir,True fir,50.127072,16.949171,0.922652,0.607735
1,2001,5,6593.333333,10.433510,None - no water sources within the accessible ...,California red fir,True fir,67.72381,19.862857,0.857143,0.742857
2,2001,7,2458.585859,12.614243,None - no water sources within the accessible ...,ponderosa pine,Oak,54.651515,13.596970,0.914141,0.464646
3,2001,9,3485.833333,6.947074,None - no water sources within the accessible ...,incense-cedar,Oak,54.354167,13.271250,0.912500,0.233333
4,2001,11,2159.259259,4.902744,Permanent streams or ponds too small to qualif...,canyon live oak,Oak,47.888889,14.200000,0.925926,0.222222


In [22]:
# pt.pivot_table(columns='species_common_name',index=['measurement_year','tree_county_code'],
#         values=['latitude','longitude','elevation','trees_per_acre_unadjusted','water_on_plot_code_name',
#                 'species_group_code_name','total_height','current_diameter','tree_status_code_name','invasive_sampling_status_code_name']).reset_index()
# piv_pt = pt.pivot_table(columns='species_group_code_name',index=['measurement_year','tree_county_code'],
#         values=['latitude','longitude','elevation','trees_per_acre_unadjusted','water_on_plot_code_name',
#                 'total_height','current_diameter','tree_status_code_name','invasive_sampling_status_code_name']).reset_index()


In [23]:
# # california plots
# # pd.read_csv('https://apps.fs.usda.gov/fia/datamart/CSV/CA_PLOT.csv')
# cap = pd.read_csv('Data/CA_PLOT.csv')

# cap_col_idx = [x-1 for x in (
#     [1]+
#     list(range(12,15))+
#     list(range(18,23))+
#     [28,54]+
#     list(range(59,64))+
#     list(range(65,70))
#     )]

# # california trees
# # pd.read_csv('https://apps.fs.usda.gov/fia/datamart/CSV/CA_TREE.csv')
# cat = pd.read_csv('Data/CA_TREE.csv',dtype={'P2A_GRM_FLG':str,'GST_PNWRS':str})

# cat_col_idx = [x-1 for x in (
#     [2,4,7]+
#     list(range(15,19))+
#     [20,21,22,24,25]+
#     list(range(145,148))
#     )]


In [24]:
# cat.columns[cat_col_idx]

In [25]:
# cap = cap[cap.columns[cap_col_idx]]
# cat = cat[cat.columns[cat_col_idx]]
# # california trees n plots merged
# capt = pd.merge(cat,cap,'left',left_on='PLT_CN',right_on='CN').drop(columns=['PLT_CN','CN'])
# capt = capt[(capt.notnull().sum() > 0)[capt.notnull().sum() > 0].index]
# # capt

In [26]:
# capt.columns

## Fires

In [27]:
conn = sqlite3.connect('Data/FPA_FOD_20221014.sqlite')

In [28]:
fire = pd.read_sql('''
                    select 
                    *
                    from fires
                    ''',conn)
fire.shape

(2303566, 39)

In [29]:
fire = pd.read_sql('''
                    select 
                    FIRE_YEAR,
                    DISCOVERY_DATE,
                    DISCOVERY_TIME,
                    NWCG_CAUSE_CLASSIFICATION,
                    NWCG_GENERAL_CAUSE,
                    FIRE_SIZE,
                    FIRE_SIZE_CLASS,
                    LATITUDE,
                    LONGITUDE,
                    FIPS_CODE,
                    FIPS_NAME
                    from fires where STATE = :state
                    ''',conn,params={'state':'CA'})
# fire = pd.read_sql('''
#                     select FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,
#                         NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,
#                         FIRE_SIZE,FIRE_SIZE_CLASS,
#                         LATITUDE,LONGITUDE
#                     from fires
#                     where STATE = :state
#                     ''',conn,params={'state':'CA'})
fire.columns = fire.columns.str.lower()

In [30]:
fire.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251881 entries, 0 to 251880
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   fire_year                  251881 non-null  int64  
 1   discovery_date             251881 non-null  object 
 2   discovery_time             206816 non-null  object 
 3   nwcg_cause_classification  251881 non-null  object 
 4   nwcg_general_cause         251881 non-null  object 
 5   fire_size                  251881 non-null  float64
 6   fire_size_class            251881 non-null  object 
 7   latitude                   251881 non-null  float64
 8   longitude                  251881 non-null  float64
 9   fips_code                  157156 non-null  object 
 10  fips_name                  157156 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 21.1+ MB


In [31]:
fire.describe().T

,count,mean,std,min,25%,50%,75%,max
fire_year,251881.0,2006.010457,8.302032,1992.000000,1999.000000,2006.000000,2013.000000,2020.0000
fire_size,251881.0,83.082573,3039.451969,0.001000,0.100000,0.200000,1.000000,589368.0000
latitude,251881.0,37.251235,2.559057,32.537406,34.661935,37.396107,39.226593,42.4175
longitude,251881.0,-120.118132,2.130694,-124.402883,-121.680684,-120.491111,-118.345556,-114.1258


In [32]:
fire

,fire_year,discovery_date,discovery_time,nwcg_cause_classification,nwcg_general_cause,fire_size,fire_size_class,latitude,longitude,fips_code,fips_name
0,2005,2/2/2005,1300,Human,Power generation/transmission/distribution,0.10,A,40.036944,-121.005833,06063,Plumas County
1,2004,5/12/2004,0845,Natural,Natural,0.25,A,38.933056,-120.404444,06061,Placer County
2,2004,5/31/2004,1921,Human,Debris and open burning,0.10,A,38.984167,-120.735556,06017,El Dorado County
3,2004,6/28/2004,1600,Natural,Natural,0.10,A,38.559167,-119.913333,06003,Alpine County
4,2004,6/28/2004,1600,Natural,Natural,0.10,A,38.559167,-119.933056,06003,Alpine County
...,...,...,...,...,...,...,...,...,...,...,...
251876,2019,10/31/2019,2115,Missing data/not specified/undetermined,Missing data/not specified/undetermined,9999.00,G,34.337222,-119.053333,06111,Ventura County
251877,2019,5/29/2019,1900,Missing data/not specified/undetermined,Missing data/not specified/undetermined,835.00,E,35.307500,-119.964444,06079,San Luis Obispo County
251878,2019,6/25/2019,2230,Missing data/not specified/undetermined,Missing data/not specified/undetermined,2422.00,F,37.472222,-121.249444,06099,Stanislaus County
251879,2019,9/4/2019,1446,Missing data/not specified/undetermined,Missing data/not specified/undetermined,54608.00,G,40.053250,-120.668900,06063,Plumas County


In [33]:
# pt['lat'] = round(pt.latitude,1)
# pt['lon'] = round(pt.longitude,1)
# fire['lat'] = round(fire.latitude,1)
# fire['lon'] = round(fire.longitude,1)
fire = fire[fire['fips_code'].notna()].copy()
fire['county_code'] = fire.fips_code.str[2:].astype(int)
fire

,fire_year,discovery_date,discovery_time,nwcg_cause_classification,nwcg_general_cause,fire_size,fire_size_class,latitude,longitude,fips_code,fips_name,county_code
0,2005,2/2/2005,1300,Human,Power generation/transmission/distribution,0.10,A,40.036944,-121.005833,06063,Plumas County,63
1,2004,5/12/2004,0845,Natural,Natural,0.25,A,38.933056,-120.404444,06061,Placer County,61
2,2004,5/31/2004,1921,Human,Debris and open burning,0.10,A,38.984167,-120.735556,06017,El Dorado County,17
3,2004,6/28/2004,1600,Natural,Natural,0.10,A,38.559167,-119.913333,06003,Alpine County,3
4,2004,6/28/2004,1600,Natural,Natural,0.10,A,38.559167,-119.933056,06003,Alpine County,3
...,...,...,...,...,...,...,...,...,...,...,...,...
251876,2019,10/31/2019,2115,Missing data/not specified/undetermined,Missing data/not specified/undetermined,9999.00,G,34.337222,-119.053333,06111,Ventura County,111
251877,2019,5/29/2019,1900,Missing data/not specified/undetermined,Missing data/not specified/undetermined,835.00,E,35.307500,-119.964444,06079,San Luis Obispo County,79
251878,2019,6/25/2019,2230,Missing data/not specified/undetermined,Missing data/not specified/undetermined,2422.00,F,37.472222,-121.249444,06099,Stanislaus County,99
251879,2019,9/4/2019,1446,Missing data/not specified/undetermined,Missing data/not specified/undetermined,54608.00,G,40.053250,-120.668900,06063,Plumas County,63


## Combo testing

In [34]:
fire.shape,pt.shape

((157156, 12), (299154, 13))

In [35]:
# fpt = pd.merge(left=fire,right=pt,how='left',left_on=['fire_year','county_code'],right_on=['measurement_year','tree_county_code'])
# fpt

In [36]:
# small_fire = fire[fire.fire_year==2018]
# small_fire

In [37]:
# small_pt = pt[pt.measurement_year==2018]
# small_pt = small_pt.dropna()
# small_pt

In [38]:
# small_fpt = pd.merge(left=small_fire,right=small_pt,how='right',left_on='county_code',right_on='tree_county_code')
# small_fpt

In [39]:
# small_aq = aq[(aq.year==2018)]
# small_aq

In [40]:
# small_aq['sample_duration'] = pd.to_numeric(small_aq['sample_duration'].str.extract('(\d+)', expand=False), errors='coerce').astype(pd.Int64Dtype())
# small_aq

In [41]:
# small_aq.info()

In [42]:
# piv_saq = small_aq.pivot_table(index=['date_local','county_code'],columns='parameter_name',
#                 values=['arithmetic_mean']).reset_index()
# piv_saq

In [43]:
# small_fpt.discovery_date = small_fpt.discovery_date.astype('datetime64[ns]')
# small_fpt

In [44]:
# piv_saq.columns

In [45]:
# small_ca = pd.merge(left=small_fpt,right=piv_saq,how='left',left_on=['discovery_date','county_code'],right_on=[('date_local',''),('county_code','')])
# small_ca

In [46]:
# small_ca.columns.to_list()

In [47]:
# small_ca.columns = [
#     'fire_year',
#     'date',
#     'time',
#     'cause_class',
#     'cause',
#     'fire_size',
#     'fire_size_class',
#     'lat',
#     'long',
#     'fips_code',
#     'county',
#     'county_code1',
#     'measurement_year',
#     'tree_county_code',
#     'latitude_y',
#     'longitude_y',
#     'elevation',
#     'trees_per_acre',
#     'water_on_plot',
#     'species',
#     'species_group',
#     'height',
#     'diameter',
#     'tree_alive',
#     'invasive_sampling',
#     'date_local',
#     'county_code2',
#     'bp_mean',
#     'co_mean',
#     'dp_mean',
#     'temp_mean',
#     'humidity_mean',
#     'wind_direction_mean',
#     'wind_speed_mean']


In [48]:
# small_ca = small_ca[[
#     'date',
#     'time',
#     'cause_class',
#     'cause',
#     'fire_size',
#     'fire_size_class',
#     'lat',
#     'long',
#     'elevation',
#     'county',
#     'trees_per_acre',
#     'water_on_plot',
#     'species',
#     'species_group',
#     'height',
#     'diameter',
#     'tree_alive',
#     'invasive_sampling',
#     'co_mean',
#     'temp_mean',
#     'humidity_mean',
#     'wind_direction_mean',
#     'wind_speed_mean']]
# small_ca.head()

In [49]:
# # Drop rows with missing data across all columns
# small_ca = small_ca.dropna()
# small_ca

In [50]:
# # map easy categorical values to 1 and 0
# small_ca.tree_alive = small_ca.tree_alive.map({'Live tree':1,'Dead tree':0})
# small_ca.invasive_sampling = small_ca.invasive_sampling.map({'Invasive plant data collected on all accessible land conditions':1,'Not collecting invasive plant data':0})
# small_ca

In [51]:
# small_ca.to_csv('ca_fire2018.csv',index=False)

In [52]:
# spt = pt[pt.measurement_year==2018]
# spt = spt.dropna()
# spt

In [53]:
# piv_spt = spt.pivot_table(index=['measurement_year','tree_county_code','latitude','longitude'],
#         columns=['species_group_code_name','water_on_plot_code_name'],
#         values=['trees_per_acre_unadjusted','total_height','current_diameter','elevation']).reset_index()
# piv_spt

## Fire and Tree Combo

In [54]:
fpt = None
for year in range(2001,2019):
    # fire
    sm_fire = fire[fire.fire_year==year]
    # tree
    sm_pt = pt_group[pt_group.measurement_year==year]
    sm_pt = sm_pt.dropna()
    # fire tree combine
    sm_fpt = pd.merge(left=sm_fire,right=sm_pt,how='left',left_on='county_code',right_on='tree_county_code')
    fpt = sm_fpt if fpt is None else pd.concat([fpt,sm_fpt],ignore_index=True)
fpt.discovery_date = fpt.discovery_date.astype('datetime64[ns]')
fpt

,fire_year,discovery_date,discovery_time,nwcg_cause_classification,nwcg_general_cause,fire_size,fire_size_class,latitude,longitude,fips_code,...,tree_county_code,elevation_mean,trees_per_acre_mean,most_common_water_source,most_common_species,most_common_species_group,height_mean,diameter_mean,percent_trees_alive,percent_invasive_plant
0,2001,2001-07-09,1515,Natural,Natural,0.2,A,39.751944,-120.402500,06091,...,91,5453.608247,8.892609,None - no water sources within the accessible ...,white fir,True fir,58.886598,15.314433,0.855670,0.824742
1,2001,2001-07-09,1630,Natural,Natural,0.1,A,39.583611,-120.418611,06091,...,91,5453.608247,8.892609,None - no water sources within the accessible ...,white fir,True fir,58.886598,15.314433,0.855670,0.824742
2,2001,2001-07-09,1715,Natural,Natural,0.2,A,39.716667,-120.433333,06091,...,91,5453.608247,8.892609,None - no water sources within the accessible ...,white fir,True fir,58.886598,15.314433,0.855670,0.824742
3,2001,2001-07-10,0945,Natural,Natural,0.3,B,39.466944,-120.350556,06091,...,91,5453.608247,8.892609,None - no water sources within the accessible ...,white fir,True fir,58.886598,15.314433,0.855670,0.824742
4,2001,2001-07-08,1115,Human,Railroad operations and maintenance,0.3,B,39.800833,-120.151111,06091,...,91,5453.608247,8.892609,None - no water sources within the accessible ...,white fir,True fir,58.886598,15.314433,0.855670,0.824742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135046,2018,2018-08-06,1459,Missing data/not specified/undetermined,Missing data/not specified/undetermined,2225.0,F,35.800000,-120.360000,06053,...,53,1691.452991,9.102330,None - no water sources within the accessible ...,redwood,Oak,76.521368,21.070940,0.777778,0.435897
135047,2018,2018-07-06,1330,Missing data/not specified/undetermined,Missing data/not specified/undetermined,1520.0,F,33.380000,-117.430000,06073,...,73,3177.419355,23.648982,None - no water sources within the accessible ...,California live oak,Oak,19.225806,9.896774,0.709677,0.096774
135048,2018,2018-06-28,1945,Missing data/not specified/undetermined,Missing data/not specified/undetermined,300.0,E,40.600000,-122.900000,06105,...,105,3912.097408,10.299555,None - no water sources within the accessible ...,Douglas-fir,Oak,61.772977,15.220738,0.785546,0.714847
135049,2018,2018-06-09,0945,Missing data/not specified/undetermined,Missing data/not specified/undetermined,175.0,D,34.360360,-118.554300,06037,...,37,4845.16129,51.612184,None - no water sources within the accessible ...,canyon live oak,Oak,18.080645,4.474194,0.693548,1.000000


## Air Pivot

In [55]:
caq = None
for year in range(2001,2019):
    # air quality
    sm_aq = aq[(aq.year==year)]
    # pivot so row is each county on date has mean of parameters
    piv_aq = sm_aq.pivot_table(index=['date_local','county_code'],columns='parameter_name',
                    values=['arithmetic_mean']).reset_index()
    caq = piv_aq if caq is None else pd.concat([caq,piv_aq],ignore_index=True)
caq

date_local county_code     arithmetic_mean                  \
parameter_name                        Barometric pressure Carbon monoxide   
0              2001-01-01           1                 NaN        1.249493   
1              2001-01-01           5                 NaN        0.612198   
2              2001-01-01           7         1011.120833        1.887802   
3              2001-01-01           9                 NaN        0.274034   
4              2001-01-01          11         1019.216667             NaN   
...                   ...         ...                 ...             ...   
313036         2018-12-31         101                 NaN             NaN   
313037         2018-12-31         107         1002.147917             NaN   
313038         2018-12-31         109                 NaN             NaN   
313039         2018-12-31         111          979.208333             NaN   
313040         2018-12-31         113                 NaN             NaN   

                                                                 \
parameter_name Dew Point Outdoor Temperature Relative Humidity    
0                    NaN                 NaN                NaN   
1                    NaN           41.708333                NaN   
2                    NaN           46.402778          77.312500   
3                    NaN           43.458333                NaN   
4                    NaN           44.270833                NaN   
...                  ...                 ...                ...   
313036               NaN           44.420833          45.100000   
313037               NaN           41.589583          77.058333   
313038               NaN           40.920833          53.462500   
313039               NaN           50.256944          46.937500   
313040               NaN           46.820833          49.400000   

                                                                  
parameter_name Wind Direction - Resultant Wind Speed - Resultant  
0                                     NaN                    NaN  
1                              155.958333               1.583333  
2                              176.625000               1.333333  
3                              160.583333               1.958333  
4                              225.625000               1.875000  
...                                   ...                    ...  
313036                         320.000000               6.225000  
313037                         140.275000               2.414583  
313038                         186.666667               1.662500  
313039                          91.923611               6.345833  
313040                         335.125000              14.983333  

[313041 rows x 9 columns]

## Fire Tree Air Combo

In [68]:
# air quality fire tree combine
ca = pd.merge(left=fpt,right=caq,how='left',left_on=['discovery_date','county_code'],right_on=[('date_local',''),('county_code','')])
ca_backup = ca

/var/folders/ml/8_q6055n29vf0c36kmj6w1900000gn/T/ipykernel_41453/1956711480.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca = pd.merge(left=fpt,right=caq,how='left',left_on=['discovery_date','county_code'],right_on=[('date_local',''),('county_code','')])


In [69]:
ca = ca_backup

In [70]:
ca.head()

,fire_year,discovery_date,discovery_time,nwcg_cause_classification,nwcg_general_cause,fire_size,fire_size_class,latitude,longitude,fips_code,...,percent_invasive_plant,"(date_local, )","(county_code, )","(arithmetic_mean, Barometric pressure)","(arithmetic_mean, Carbon monoxide)","(arithmetic_mean, Dew Point)","(arithmetic_mean, Outdoor Temperature)","(arithmetic_mean, Relative Humidity )","(arithmetic_mean, Wind Direction - Resultant)","(arithmetic_mean, Wind Speed - Resultant)"
0,2001,2001-07-09,1515,Natural,Natural,0.2,A,39.751944,-120.402500,06091,...,0.824742,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001,2001-07-09,1630,Natural,Natural,0.1,A,39.583611,-120.418611,06091,...,0.824742,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2001,2001-07-09,1715,Natural,Natural,0.2,A,39.716667,-120.433333,06091,...,0.824742,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2001,2001-07-10,0945,Natural,Natural,0.3,B,39.466944,-120.350556,06091,...,0.824742,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2001,2001-07-08,1115,Human,Railroad operations and maintenance,0.3,B,39.800833,-120.151111,06091,...,0.824742,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
ca.columns

Index([                                      'fire_year',
                                        'discovery_date',
                                        'discovery_time',
                             'nwcg_cause_classification',
                                    'nwcg_general_cause',
                                             'fire_size',
                                       'fire_size_class',
                                              'latitude',
                                             'longitude',
                                             'fips_code',
                                             'fips_name',
                                           'county_code',
                                      'measurement_year',
                                      'tree_county_code',
                                        'elevation_mean',
                                   'trees_per_acre_mean',
                              'most_common_water_source',
              

In [72]:
# rename for readability and ease of use
ca.columns = [
    'fire_year',
    'date',
    'time',
    'cause_class',
    'cause',
    'fire_size',
    'fire_size_class',
    'lat',
    'long',
    'fips_code',
    'county',
    'county_code1',
    'measurement_year',
    'tree_county_code',
    'elevation_mean',
    'trees_per_acre_mean',
    'most_common_water_source',
    'most_common_species',
    'most_common_species_group',
    'height_mean',
    'diameter_mean',
    'percent_trees_alive',
    'percent_invasive_plant',
    'date_local',
    'county_code2',
    'bp_mean',
    'co_mean',
    'dp_mean',
    'temp_mean',
    'humidity_mean',
    'wind_direction_mean',
    'wind_speed_mean']
# keep necessary columns, drop others
ca = ca[[
    'date',
    'time',
    'cause_class',
    'cause',
    'fire_size',
    'fire_size_class',
    'lat',
    'long',
    'elevation_mean',
    'county',
    'trees_per_acre_mean',
    'most_common_water_source',
    'most_common_species',
    'most_common_species_group',
    'height_mean',
    'diameter_mean',
    'percent_trees_alive',
    'percent_invasive_plant',
    'co_mean',
    'temp_mean',
    'humidity_mean',
    'wind_direction_mean',
    'wind_speed_mean']]
ca.shape

(135051, 23)

In [73]:
ca.isnull().sum()

date                             0
time                           331
cause_class                      0
cause                            0
fire_size                        0
fire_size_class                  0
lat                              0
long                             0
elevation_mean               12175
county                           0
trees_per_acre_mean          12175
most_common_water_source     12175
most_common_species          12175
most_common_species_group    12175
height_mean                  12175
diameter_mean                12175
percent_trees_alive          12175
percent_invasive_plant       12175
co_mean                      53596
temp_mean                    32329
humidity_mean                53969
wind_direction_mean          42455
wind_speed_mean              41903
dtype: int64

In [74]:
ca.shape

(135051, 23)

In [75]:
# Drop rows with missing data
# ca[(ca.co_mean.notna())&(ca.wind_speed_mean.notna())&(ca.humidity_mean.notna())&(ca.wind_direction_mean.notna())&(ca.time.notna())].isnull().sum()
ca = ca.dropna()
ca.shape

(49243, 23)

In [62]:
# ca.date.dt.year.value_counts().sort_index()

In [63]:
# map easy categorical values to 1 and 0
# ca.tree_alive = ca.tree_alive.map({'Live tree':1,'Dead tree':0})
# ca.invasive_sampling = ca.invasive_sampling.map({'Invasive plant data collected on all accessible land conditions':1,'Not collecting invasive plant data':0})
# ca.sample(5)

In [76]:
# make datetime to create features
ca.date = ca.date.astype('datetime64[ns]')
# fix dtype
ca.most_common_water_source = ca.most_common_water_source.astype(str)
# get unique values for renaming
water = ca.most_common_water_source.value_counts().sort_index().index.to_list()
water

['Flood zones - evidence of flooding WHEN water_on_plot_code = bodies of water exceed their natural banks',
 'None - no water sources within the accessible forest land condition class',
 'Other temporary water',
 'Permanent streams or ponds too small to qualify as noncensus water',
 'Temporary streams']

In [77]:
# rename values
ca.most_common_water_source = ca.most_common_water_source.map({
    water[0]:'flood_zone',
    water[1]:'none',
    water[2]:'temp_water',
    water[3]:'small_water',
    water[4]:'temp_water'})
# make all values lowercase
for col in ca.select_dtypes(include=('object')).columns:
    ca[col] = ca[col].str.lower()
# make features
ca['month'] = ca.date.dt.month.copy()
ca['day_of_year'] = ca.date.dt.dayofyear.copy()

In [78]:
ca

,date,time,cause_class,cause,fire_size,fire_size_class,lat,long,elevation_mean,county,...,diameter_mean,percent_trees_alive,percent_invasive_plant,co_mean,temp_mean,humidity_mean,wind_direction_mean,wind_speed_mean,month,day_of_year
23,2001-07-26,0500,human,missing data/not specified/undetermined,0.2,a,39.017778,-120.716667,5350.684932,placer county,...,15.169863,0.851598,0.591324,0.276540,75.562500,53.166667,164.541667,4.645834,7,207
31,2001-09-08,2100,human,other causes,0.2,a,39.169167,-120.218056,5350.684932,placer county,...,15.169863,0.851598,0.591324,0.316666,72.354166,45.625000,163.208333,3.479166,9,251
34,2001-07-23,1730,natural,natural,1.0,b,39.167778,-120.250278,5350.684932,placer county,...,15.169863,0.851598,0.591324,0.269318,79.166666,41.083333,184.270833,3.020833,7,204
43,2001-04-01,1400,human,smoking,0.2,a,38.883889,-120.135278,5033.219178,el dorado county,...,15.157706,0.895548,0.635274,0.262107,46.555556,52.250000,176.930556,3.805556,4,91
44,2001-06-25,0430,human,smoking,0.1,a,38.851944,-120.017778,5033.219178,el dorado county,...,15.157706,0.895548,0.635274,0.203442,59.854167,35.541667,184.979167,3.552084,6,176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135041,2018-06-12,1630,missing data/not specified/undetermined,missing data/not specified/undetermined,154.0,d,33.930000,-117.660000,6924.299065,san bernardino county,...,17.179439,0.728972,0.654206,0.296034,79.469444,32.193452,207.838068,3.942898,6,163
135044,2018-07-13,0045,missing data/not specified/undetermined,missing data/not specified/undetermined,962.0,e,39.790000,-121.740000,3473.752311,butte county,...,15.395564,0.887246,0.415896,0.260428,81.318055,45.697222,158.375000,1.794444,7,194
135045,2018-08-03,1648,missing data/not specified/undetermined,missing data/not specified/undetermined,3516.0,f,35.400000,-118.770000,5000.3367,kern county,...,10.737710,0.730640,0.484848,0.256431,87.292083,27.532143,221.747917,5.087500,8,215
135047,2018-07-06,1330,missing data/not specified/undetermined,missing data/not specified/undetermined,1520.0,f,33.380000,-117.430000,3177.419355,san diego county,...,9.896774,0.709677,0.096774,0.528450,84.014286,37.200000,203.154167,3.379028,7,187


In [80]:
# make datetime
ca.date = ca.date.astype('datetime64[ns]')

In [81]:
ca.to_csv('ca_fire.csv',index=False)

In [2]:
import big_wrangle as w
import numpy as np
import pandas as pd

In [7]:
fires = w.wrangle_fires()
forest = w.get_forest()

In [9]:
forest.water_on_plot_code_name.value_counts()

None - no water sources within the accessible forest land condition class                                                                                 213452
Temporary streams                                                                                                                                          47369
Permanent streams or ponds too small to qualify as noncensus water                                                                                         32030
Other temporary water                                                                                                                                       2606
                                                                                                                                                            1438
Permanent water in the form of deep swamps, bogs, marshes without standing trees present and less than 1.0 acre in size, or with standing trees              956
Ditch/canal - human-made channels 

In [10]:
def prep_forest(pt):
    '''Prepare forest data'''
    # I use pt because usfs table was called plot_tree
    # drop them na trees
    pt = pt[pt['tree_county_code'].notna()].copy()
    # map easy categorical values to 1 and 0
    pt.tree_status_code_name = pt.tree_status_code_name.map({'Live tree':1,'Dead tree':0})
    pt.invasive_sampling_status_code_name = pt.invasive_sampling_status_code_name.map({'Invasive plant data collected on all accessible land conditions':1,'Not collecting invasive plant data':0})
    # fix dtype
    pt.water_on_plot_code_name = pt.water_on_plot_code_name.astype(str)
    # boolean water values
    pt.water_on_plot_code_name = pt.water_on_plot_code_name.map({
        'Flood zones - evidence of flooding WHEN water_on_plot_code = bodies of water exceed their natural banks':1,
        'None - no water sources within the accessible forest land condition class':0,
        'Other temporary water':1,
        'Permanent streams or ponds too small to qualify as noncensus water':1,
        'Temporary streams':1,
        '':0,
        'Ditch/canal - human-made channels used as a means of moving water, e.g., for irrigation or drainage, which are too small to qualify as noncensus water':1,
        'Permanent water in the form of deep swamps, bogs, marshes without standing trees present and less than 1.0 acre in size, or with standing trees':1})
    # Performed 9 aggregations grouped on columns: 'measurement_year', 'tree_county_code'
    # used for merge so each row has most common tree and stats for county and year
    return pt.groupby(
        ['measurement_year', 'tree_county_code']
            ).agg(
                elevation_mean=('elevation', 'mean'), 
                trees_per_acre_mean=('trees_per_acre_unadjusted', 'mean'),
                percent_chance_water_nearby=('water_on_plot_code_name', 'mean'),
                most_common_species=('species_common_name', lambda s: s.value_counts().index[0]),
                most_common_species_group=('species_group_code_name', lambda s: s.value_counts().index[0]),
                height_mean=('total_height', 'mean'),
                diameter_mean=('current_diameter', 'mean'),
                percent_trees_alive=('tree_status_code_name', 'mean'),
                percent_invasive_plant=('invasive_sampling_status_code_name', 'mean')
                ).reset_index()
forest = prep_forest(forest)
forest

,measurement_year,tree_county_code,elevation_mean,trees_per_acre_mean,percent_chance_water_nearby,most_common_species,most_common_species_group,height_mean,diameter_mean,percent_trees_alive,percent_invasive_plant
0,2001,3,8207.18232,8.794056,0.662983,California red fir,True fir,50.127072,16.949171,0.922652,0.607735
1,2001,5,6593.333333,10.433510,0.209524,California red fir,True fir,67.72381,19.862857,0.857143,0.742857
2,2001,7,2458.585859,12.614243,0.247475,ponderosa pine,Oak,54.651515,13.596970,0.914141,0.464646
3,2001,9,3485.833333,6.947074,0.254167,incense-cedar,Oak,54.354167,13.271250,0.912500,0.233333
4,2001,11,2159.259259,4.902744,0.777778,canyon live oak,Oak,47.888889,14.200000,0.925926,0.222222
...,...,...,...,...,...,...,...,...,...,...,...
888,2018,107,7432.475884,6.126898,0.342444,white fir,True fir,67.673633,20.714309,0.734727,0.628617
889,2018,109,6629.251701,10.894003,0.178571,California red fir,True fir,68.807823,19.636395,0.797619,0.336735
890,2018,111,4932.380952,10.840981,0.352381,canyon live oak,Oak,28.780952,9.653333,0.628571,1.000000
891,2018,113,1887.5,6.018046,0.312500,blue oak,Oak,28.9375,9.768750,1.000000,0.000000


In [11]:
forest.most_common_species_group.value_counts()

Oak                            376
True fir                       176
Other western hardwoods        115
Woodland softwoods              55
Ponderosa and Jeffrey pines     45
Douglas-fir                     44
Redwood                         26
Other western softwoods         19
Lodgepole pine                  14
Woodland hardwoods              13
Incense-cedar                    6
Cottonwood and aspen             2
Sugar pine                       1
Western white pine               1
Name: most_common_species_group, dtype: int64

In [2]:
fpt = w.wrangle_forest_fires()
caq = w.wrangle_air_quality()

# forest fire and air data merge on data and county to forest fire
ca = pd.merge(left=fpt,right=caq,how='left',
                left_on=['discovery_date','county_code'],
                right_on=[('date_local',''),('county_code','')])
# rename for readability and ease of use
ca.columns = [
    'fire_year',
    'date',
    'time',
    'cause_class',
    'cause',
    'fire_size',
    'fire_size_class',
    'lat',
    'long',
    'fips_code',
    'county',
    'county_code1',
    'measurement_year',
    'tree_county_code',
    'elevation_mean',
    'trees_per_acre_mean',
    'percent_chance_water_nearby',
    'most_common_species',
    'most_common_species_group',
    'height_mean',
    'diameter_mean',
    'percent_trees_alive',
    'percent_invasive_plant',
    'date_local',
    'county_code2',
    'bp_mean',
    'co_mean',
    'dp_mean',
    'temp_mean',
    'humidity_mean',
    'wind_direction_mean',
    'wind_speed_mean']
# keep necessary columns, drop others
ca = ca[[
    'date',
    'time',
    'cause_class',
    'cause',
    'fire_size',
    'fire_size_class',
    'lat',
    'long',
    'elevation_mean',
    'county',
    'trees_per_acre_mean',
    'percent_chance_water_nearby',
    'most_common_species_group',
    'height_mean',
    'diameter_mean',
    'percent_trees_alive',
    'percent_invasive_plant',
    'co_mean',
    'temp_mean',
    'humidity_mean',
    'wind_direction_mean',
    'wind_speed_mean']]
# get rid of nulls
ca = ca.dropna()
# make datetime to create features
ca.date = ca.date.astype('datetime64[ns]')
# make all values lowercase
for col in ca.select_dtypes(include=('object')).columns:
    ca[col] = ca[col].str.lower()
# make time features
ca['month'] = ca.date.dt.month.copy()
ca['day_of_year'] = ca.date.dt.dayofyear.copy()
# bin counties
ca.county = ca.county.str.replace(' county','')
jefferson = ['butte', 'colusa', 'del norte', 'glenn', 'humboldt', 'lake', 'lassen', 'mendocino', 'modoc', 'plumas', 'shasta', 'siskiyou', 'tehama', 'trinity']
north_cali = ['amador', 'el dorado', 'marin', 'napa', 'nevada', 'placer', 'sacramento', 'sierra', 'solano', 'sonoma', 'sutter', 'yolo', 'yuba']
silicon_valley = ['alameda', 'contra costa', 'monterey', 'san benito', 'san francisco', 'san mateo', 'santa clara', 'santa cruz']
central_cali = ['alpine', 'calaveras', 'fresno', 'inyo', 'kern', 'kings', 'madera', 'mariposa', 'merced', 'mono', 'san joaquin', 'stanislaus', 'tulare', 'tuolumne']
west_cali = ['los angeles', 'san luis obispo', 'santa barbara', 'ventura']
south_cali = ['imperial', 'orange', 'riverside', 'san bernardino', 'san diego']
ca['six_cali'] = np.where(ca.county.isin(jefferson),'jefferson','')
ca['six_cali'] = np.where(ca.county.isin(north_cali),'north_cali',ca['six_cali'])
ca['six_cali'] = np.where(ca.county.isin(silicon_valley),'silicon_valley',ca['six_cali'])
ca['six_cali'] = np.where(ca.county.isin(central_cali),'central_cali',ca['six_cali'])
ca['six_cali'] = np.where(ca.county.isin(west_cali),'west_cali',ca['six_cali'])
ca['six_cali'] = np.where(ca.county.isin(south_cali),'south_cali',ca['six_cali'])
# bin species group
willow = ['cottonwood and aspen']
pine = ['douglas-fir','lodgepole pine','ponderosa and jeffrey pines','true fir']
cypress = ['redwood','incense-cedar']
softwoods = ['woodland softwoods','other western softwoods']
hardwoods = ['woodland hardwoods','other western hardwoods']
oak = ['oak'] # I will just leave as oak
ca['tree_family'] = np.where(ca.most_common_species_group.isin(oak),'oak','')
ca['tree_family'] = np.where(ca.most_common_species_group.isin(hardwoods),'hardwood',ca['tree_family'])
ca['tree_family'] = np.where(ca.most_common_species_group.isin(softwoods),'softwood',ca['tree_family'])
ca['tree_family'] = np.where(ca.most_common_species_group.isin(cypress),'cypress',ca['tree_family'])
ca['tree_family'] = np.where(ca.most_common_species_group.isin(pine),'pine',ca['tree_family'])
ca['tree_family'] = np.where(ca.most_common_species_group.isin(willow),'willow',ca['tree_family'])
# cache locally
ca.to_csv('ca_fire.csv',index=False)

/var/folders/ml/8_q6055n29vf0c36kmj6w1900000gn/T/ipykernel_65297/1976911708.py:5: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca = pd.merge(left=fpt,right=caq,how='left',


In [3]:
d = w.wrangle_wildfires()
d

/Users/Tyler/GitHub/codeup-data-science/ca_wildfires/big_wrangle.py:288: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  ca = pd.merge(left=fpt,right=caq,how='left',


,date,time,cause_class,cause,fire_size,fire_size_class,lat,long,elevation_mean,county,...,percent_invasive_plant,co_mean,temp_mean,humidity_mean,wind_direction_mean,wind_speed_mean,month,day_of_year,six_cali,most_common_is_hardwood
23,2001-07-26,0500,human,missing data/not specified/undetermined,0.20,a,39.017778,-120.716667,5350.684932,placer,...,0.591324,0.276540,75.562500,53.166667,164.541667,4.645834,7,207,north_cali,0
31,2001-09-08,2100,human,other causes,0.20,a,39.169167,-120.218056,5350.684932,placer,...,0.591324,0.316666,72.354166,45.625000,163.208333,3.479166,9,251,north_cali,0
34,2001-07-23,1730,natural,natural,1.00,b,39.167778,-120.250278,5350.684932,placer,...,0.591324,0.269318,79.166666,41.083333,184.270833,3.020833,7,204,north_cali,0
43,2001-04-01,1400,human,smoking,0.20,a,38.883889,-120.135278,5033.219178,el dorado,...,0.635274,0.262107,46.555556,52.250000,176.930556,3.805556,4,91,north_cali,0
44,2001-06-25,0430,human,smoking,0.10,a,38.851944,-120.017778,5033.219178,el dorado,...,0.635274,0.203442,59.854167,35.541667,184.979167,3.552084,6,176,north_cali,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134670,2018-11-19,0444,undetermined,missing data/not specified/undetermined,0.01,a,37.364405,-121.977526,1805.825243,santa clara,...,0.000000,1.409760,52.541667,63.916667,204.820833,2.183333,11,323,silicon_valley,1
135039,2018-06-11,1849,undetermined,missing data/not specified/undetermined,155.00,d,33.750000,-117.720000,900.0,orange,...,0.000000,0.258333,71.917708,65.166667,227.010417,2.883333,6,162,south_cali,1
135041,2018-06-12,1630,undetermined,missing data/not specified/undetermined,154.00,d,33.930000,-117.660000,6924.299065,san bernardino,...,0.654206,0.296034,79.469444,32.193452,207.838068,3.942898,6,163,south_cali,0
135044,2018-07-13,0045,undetermined,missing data/not specified/undetermined,962.00,e,39.790000,-121.740000,3473.752311,butte,...,0.415896,0.260428,81.318055,45.697222,158.375000,1.794444,7,194,jefferson,1
